In [2]:
import pandas as pd

In [3]:
df_search_product = pd.read_csv("/home/music/Downloads/view_b_product.csv")

In [108]:
df_search_product['BID'] = df_search_product['BID'].fillna(-1).astype(int)

In [4]:
ls_searh_field = ['PRODUCT_NAME', 'PREVIEW_TEXT', 'TAGS', 'BRAND_NAME', 'TAGS_NAME']

In [40]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [139]:
stop_lang_filter = {
    "stop_lang": {
    "type":       "stop",
    "stopwords":  [
        "_thai_" ,
        "_english_" 
        ]
    }
}

english_stemmer_filter = {
    "english_stemmer": {
        "type":       "stemmer",
        "language":   "english"
    }
}

english_possessive_stemmer_filter = {
    "english_possessive_stemmer": {
        "type":       "stemmer",
        "language":   "possessive_english"
    }
}

ls_filter = [
    "lowercase",
    "decimal_digit",
    "stop_lang",
    "asciifolding",
    "classic",
    "english_stemmer",
    "english_possessive_stemmer"
]

search = {
    "search": {
        "tokenizer":  "classic",
        "filter": [
            "lowercase",
            "decimal_digit",
            "asciifolding",
            "classic",
        ]
    }
}

full_search = {
    "full_search": {
        "tokenizer":  "classic",
        "filter": [
            "lowercase",
            "decimal_digit",
            "stop_lang",
            "asciifolding",
            "classic",
            "english_stemmer",
            "english_possessive_stemmer"
        ]
    }
}

full_put_search = {
    "full_put_search": {
        "tokenizer":  "classic",
        "filter": [
            "lowercase",
            "decimal_digit",
            "stop_lang",
            "asciifolding",
            "classic",
            "english_stemmer",
            "english_possessive_stemmer",
            "unique"
        ]
    }
}

put_search = {
    "put_search": {
        "tokenizer":  "classic",
        "filter": [
            "lowercase",
            "decimal_digit",
            "asciifolding",
            "classic",
            "unique"
        ]
    }
}

setting = {
    "settings": {
        "analysis": {
            "filter": {
                **stop_lang_filter, 
                **english_stemmer_filter, 
                **english_possessive_stemmer_filter
            },
            "analyzer": {
                **search,
                **put_search,
                **full_put_search,
                **full_search
            }
        }
    }
}

In [143]:
mapping = {
    "mappings":{
        "properties":{
            "search":{
                "type":"text",
                "analyzer":"put_search",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    },
                    "full_put_search": {
                        "type": "text",
                        "analyzer": "full_put_search"
                    },
                    "search": {
                        "type": "text",
                        "analyzer": "search"
                    },
                    "full_search": {
                        "type": "text",
                        "analyzer": "full_search"
                    }
                }
            },
            'product_id':{
                "type":"keyword",
            },
            'original_search':{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
                    
            },
            'product_name':{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            'brand_name':{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            'brand_id':{
                "type":"keyword",
            },
        }
    }
}

In [59]:
es = Elasticsearch("192.168.1.97:9200")

In [144]:
name_index = "search_product"
if not es.indices.exists(index=name_index):
    es.indices.create(index=name_index, body={**setting, **mapping})

/tmp/ipykernel_19889/1457842971.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=name_index, body={**setting, **mapping})


In [103]:
from tqdm import tqdm
import numpy as np

In [145]:
ls_bulks = []

for row in tqdm(range(len(df_search_product))):
    data = df_search_product.iloc[row]
    str_search = []
    for field in ls_searh_field:
        text = data[field]
        if pd.isna(text):
            continue
        elif type(text) != str:
            text = str(text)
        str_search.append(text)
        
    str_search = ' '.join(str_search)
    body = {
        '_id': row,
        "search": str_search,
        "product_id": data['EID'],
        'original_search': str_search,
        'product_name': data['PRODUCT_NAME'],
        'brand_name': '' if data['BRAND_NAME'] is np.nan else data['BRAND_NAME'],
        'brand_id': -1 if data['BID'] is np.nan else data['BID'],
    }
    ls_bulks.append(body)
    if row % 1000 == 999:
        bulk(es, ls_bulks, index=name_index)
        ls_bulks = []


100%|██████████| 30030/30030 [00:19<00:00, 1557.12it/s]


In [149]:
def auto_complete(text):
    analyzed_text = es.indices.analyze(index=name_index, body={"analyzer": "search", "text": text})['tokens']
    body_search = {'suggest': {}}
    for n, text in enumerate(analyzed_text):
        body_search['suggest'][str(n)+'_search'] = {}
        body_search['suggest'][str(n)+'_search']['text'] = text['token']
        body_search['suggest'][str(n)+'_search']['term'] = {
            "field": "search",
            "size": 1,
            "min_word_length" : 2}
    return es.search(index=name_index, body=body_search)['suggest']

def search_product(ls_suggest, n_result=5):
    ls_text = []
    for n, text in enumerate(ls_suggest):
        if len(ls_suggest[text][0]['options']) > 0:
            ls_text.append(ls_suggest[text][0]['options'][0]['text'])
        else:
            
            ls_text.append(ls_suggest[text][0]['text'])
            
    body_product = {
        "query": {
            "match": {
                "search": { 
                    "query": ' '.join(ls_text),
                    "analyzer": "search"
                }
            }
        },
        "_source": {
            "excludes": ['original_search', 'brand_id']
        },
        "size": n_result
    }
    return es.search(index=name_index, body=body_product)

In [150]:
def pipeline_search(text, n_result=5):
    ls_suggest = auto_complete(text)
    result = search_product(ls_suggest, n_result=n_result)
    ls_product = []
    for res in result['hits']['hits']:
        print(f"{res['_score']:.2f} : {res['_source']['brand_name']} : {res['_source']['product_name']}")
        print(f"    {res['_source']['search']}")
        print("=====================================")
        ls_product.append(res['_source']['product_id'])
    return ls_product

In [187]:
import time

In [191]:
test_text = "นีเวีย ชาย"
st = time.time()
ls_product = pipeline_search(test_text, n_result=10)
print(f"time {(time.time() - st)*1000:.2f} ms")

10.27 : Ranong Tea : Ranongtea Herbalist
    Ranongtea Herbalist Ranongtea Herbalist, เครื่องดื่มเพื่อสุขภาพ, ชา, tea Ranong Tea Dietary Food (Healthy care), ผลิตภัณฑ์เสริมอาหาร, อาหารเสริม, อาหารเสริมบำรุงสมอง, วิตามินบำรุงสายตา, วิตามินบำรุงร่างกาย
6.42 : NIVEA : Original Care
    Original Care Nivea Original Care NIVEA ORIGINAL CARE, NIVEA ESSENTIAL CARE, NIVEA, LIP CARE, ลิปมัน, นีเวีย, ลิปมัน นีเวีย NIVEA Lip Care, ลิปแคร์, ลิปมัน, ลิปบาล์ม, ลิปบำรุงริมฝีปาก, ลิปมันเปลี่ยนสี
6.24 : NIVEA : White Day Cream SPF15
    White Day Cream SPF15 Nivea White Day Cream SPF15 nivea, day cream, face cream, cream, นีเวีย, เดย์ครีม, ครีมบำรุงผิวหน้า NIVEA Day Cream, เดย์ครีม, ครีมบํารุงผิวหน้ากลางวัน, ครีมทากลางวัน, ครีมกลางวัน
6.24 : NIVEA : Lip Butter
    Lip Butter Nivea Lip Butter nivea, nivea lip butter, lip butter, lip care, นีเวีย, ลิป บัตเตอร์, ลิปแคร์, ลิป NIVEA Lip Care, ลิปแคร์, ลิปมัน, ลิปบาล์ม, ลิปบำรุงริมฝีปาก, ลิปมันเปลี่ยนสี
6.24 : NIVEA : Micellair Expert Micellar Wipes
    Mice

/tmp/ipykernel_19889/2998513151.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  return es.search(index=name_index, body=body_search)['suggest']
/tmp/ipykernel_19889/2998513151.py:36: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  return es.search(index=name_index, body=body_product)


In [184]:
ls_product

[35208, 647, 644, 2434, 646, 643, 14625, 653, 2269, 39718]

In [154]:
df_rev = pd.read_csv("/home/music/Downloads/review.csv", low_memory=False)
df_rev.columns

Index(['ID', 'EID', 'BID', 'ACTION', 'COMMENT', 'REGULAR', 'ACTUAL',
       'DATE_CREATE', 'modify_date', 'TIMESTAMP_TOP', 'UID', 'AUTHOR_IP',
       'AUTHOR_REAL_IP', 'COMMENT_IMG', 'COMMENT_IMG_2', 'COMMENT_IMG_3',
       'COMMENT_IMG_4', 'COMMENT_IMG_5', 'REVIEW_SHOW', 'REVIEW_WHEN', 'POINT',
       'ENAME', 'EIMG', 'BRAND', 'TYPE', 'TYPE_ID', 'UNAME', 'UIMG',
       'LOCATION', 'EFFECT', 'D_USER', 'FID', 'campaing_name', 'SEARCH_TXT'],
      dtype='object')

In [161]:
df_rev.iloc[0]['COMMENT']

'ดินสอเขียนคิ้ว เมลินดา ตัวนี้ใช้มาหลายแท่งมากๆ เป็นสิบแท่งก็ว่าได้ แต่เพิ่งจะมีโอกาสมารีวิว เนื้อดินสอนุ่ม เขียนง่ายมาก สีสวย กันน้ำ กันเหงื่อ ติดทนนานมากๆค่ะ เพื่อนๆคนไหนกำลังมองหาดินสอเขียนคิ้วถูกและดี แนะนำตัวนี่เลยค่ะ เริ่ดมากค่า'

In [185]:
review = '\n\n'.join(df_rev[df_rev['EID'] == ls_product[1]]['COMMENT'].values.tolist())
ls_review = df_rev[df_rev['EID'] == ls_product[1]]['COMMENT'].values.tolist()
review_clean = '\n\n'.join([review.replace('\r', ' ') for review in ls_review])

In [186]:
print(review_clean)

กลิ่นหอมติดทนนานมาก ติดทนนานทั้งวันเลย เป็นกลิ่นแบบผู้ใหญ่นิดๆ ใช้ได้เรื่อๆไม่ฉุนเลย

ได้ไซส์เทสเตอร์มาลองใช้ดูก่อนค่ะ ก่อนนี้ไม่เคยใช้น้ำหอมเลยอยากลองใช้ดูบ้าง เลยลองเลือกแบรนด์นี้มาลอง กลิ่นหอมติดทนนานมากค่ะ ตั้งแต่เช้าจรดเย็นกลิ่นไม่จางเลย ชอบที่กลิ่นหอมกำลังดี จะว่าหวานก็หวานแต่หวานไม่มาก แอบมีความเซ็กซี่เล็กๆ ไม่ชวนเวียนหัวดีค่ะ เราว่าจะฉีดในโอกาสไหนๆก็ได้ทั้งนั้นนะ ทั้งทำงาน เที่ยว หรือโอกาสพิเศษอื่นๆ

หอมมากๆ  หอมแนวไฮโซๆ
